In [1]:
import json, pickle

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer

In [104]:
# Reading json file
intents = json.loads(open('chats.json').read())

In [105]:
words = []
classes = []
documents = []
ignore_letters = ['?', ',', '|', '.', '/', '\\', '#', '!', '^', '$']

In [106]:
# splitting the words
for i in intents['intents']:
    for p in i['patterns']:
        w = nltk.word_tokenize(p)
        words.extend(w)
        documents.append((w, i['tags']))
        
        if i['tags'] not in classes:
            classes.append(i['tags'])

In [107]:
# Removing unwanted char

In [108]:
lem = WordNetLemmatizer()

In [109]:
words = sorted(set([lem.lemmatize(w) for w in words if w not in ignore_letters]))
classes = sorted(set(classes))

In [110]:
# Saving the words to file
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [111]:
# Training data

In [112]:
training = []
output = [0] * len(classes)

In [113]:
for d in documents:
    bag = []
    wPat = d[0]
    wPat = [lem.lemmatize(w.lower()) for w in wPat]
    
    for w in words:
        bag.append(1) if w in wPat else bag.append(0)
    
    outputR = list(output)
    outputR[classes.index(d[1])] = 1
    training.append([bag, outputR])

In [114]:
import random
import numpy as np

In [115]:
random.shuffle(training)
training = np.array(training)

In [116]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [117]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

In [118]:
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

In [119]:
from tensorflow.keras.optimizers import SGD

In [120]:
sgd = SGD(lr = 0.01, momentum = 0.9, nesterov = True, decay = 1e-6)

In [121]:
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [122]:
train = model.fit(np.array(train_x), np.array(train_y), batch_size = 5, epochs = 200, verbose = 1)
print('\nDone training!')

Epoch 1/200
22/22 [==============================] - 0s 4ms/step - loss: 2.2679 - accuracy: 0.2075
Epoch 2/200
22/22 [==============================] - 0s 909us/step - loss: 2.1378 - accuracy: 0.3019
Epoch 3/200
22/22 [==============================] - 0s 950us/step - loss: 2.0670 - accuracy: 0.2925
Epoch 4/200
22/22 [==============================] - 0s 923us/step - loss: 2.0352 - accuracy: 0.3019
Epoch 5/200
22/22 [==============================] - 0s 1ms/step - loss: 1.9645 - accuracy: 0.3019
Epoch 6/200
22/22 [==============================] - 0s 993us/step - loss: 1.9040 - accuracy: 0.3491
Epoch 7/200
22/22 [==============================] - 0s 1ms/step - loss: 1.8307 - accuracy: 0.3302
Epoch 8/200
22/22 [==============================] - 0s 1ms/step - loss: 1.8081 - accuracy: 0.3774
Epoch 9/200
22/22 [==============================] - 0s 1ms/step - loss: 1.7100 - accuracy: 0.4245
Epoch 10/200
22/22 [==============================] - 0s 1ms/step - loss: 1.7085 - accuracy: 0.4434
E

22/22 [==============================] - 0s 904us/step - loss: 0.3090 - accuracy: 0.9057
Epoch 82/200
22/22 [==============================] - 0s 786us/step - loss: 0.3653 - accuracy: 0.9151
Epoch 83/200
22/22 [==============================] - 0s 966us/step - loss: 0.3732 - accuracy: 0.8774
Epoch 84/200
22/22 [==============================] - 0s 901us/step - loss: 0.3420 - accuracy: 0.8868
Epoch 85/200
22/22 [==============================] - 0s 761us/step - loss: 0.3819 - accuracy: 0.8774
Epoch 86/200
22/22 [==============================] - 0s 807us/step - loss: 0.4082 - accuracy: 0.8868
Epoch 87/200
22/22 [==============================] - 0s 807us/step - loss: 0.3298 - accuracy: 0.8962
Epoch 88/200
22/22 [==============================] - 0s 891us/step - loss: 0.3250 - accuracy: 0.9151
Epoch 89/200
22/22 [==============================] - 0s 760us/step - loss: 0.2736 - accuracy: 0.9245
Epoch 90/200
22/22 [==============================] - 0s 838us/step - loss: 0.3718 - accuracy: 

22/22 [==============================] - 0s 757us/step - loss: 0.2207 - accuracy: 0.9245
Epoch 161/200
22/22 [==============================] - 0s 797us/step - loss: 0.2781 - accuracy: 0.8962
Epoch 162/200
22/22 [==============================] - 0s 795us/step - loss: 0.2554 - accuracy: 0.9057
Epoch 163/200
22/22 [==============================] - 0s 828us/step - loss: 0.2773 - accuracy: 0.8962
Epoch 164/200
22/22 [==============================] - 0s 836us/step - loss: 0.2511 - accuracy: 0.9057
Epoch 165/200
22/22 [==============================] - 0s 786us/step - loss: 0.2379 - accuracy: 0.9057
Epoch 166/200
22/22 [==============================] - 0s 750us/step - loss: 0.2324 - accuracy: 0.9245
Epoch 167/200
22/22 [==============================] - 0s 736us/step - loss: 0.2157 - accuracy: 0.9245
Epoch 168/200
22/22 [==============================] - 0s 754us/step - loss: 0.2439 - accuracy: 0.9151
Epoch 169/200
22/22 [==============================] - 0s 712us/step - loss: 0.2023 - a

In [123]:
model.save('chatbot_trained_model.h5', train)